In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
from spherical_coordinates import *

In [41]:
#Load data
directory_2D = "D:\\CTA data\\Segments_deformed_2\\"
file_names_2D = os.listdir(directory_2D)
directory_3D = "D:\\CTA data\\Segments renamed\\"
file_names_3D = os.listdir(directory_3D)

data_2d = np.genfromtxt(os.path.join(directory_2D, file_names_2D[0]), delimiter=",")
data_3d = np.genfromtxt(os.path.join(directory_3D, file_names_3D[0]), delimiter=",")[1:, 1:4]

#Add row of zeros
data_2d = np.hstack((data_2d, np.zeros((data_2d.shape[0],1))))

#Normalize 3D
data_3d[:, 0] -= np.min(data_3d[:, 0])
data_3d[:, 1] -= np.min(data_3d[:, 1])
data_3d[:, 2] -= np.min(data_3d[:, 2])

#Convert to spherical:
origin_2D, spherical_2D = convert_to_spherical(data_2d)
origin_3D, spherical_3D = convert_to_spherical(data_3d)
spherical_2D

tensor([[0.0228, 1.5708, 0.8024],
        [0.0235, 1.5708, 0.8314],
        [0.0228, 1.5708, 0.7965],
        ...,
        [0.0234, 1.5708, 0.3989],
        [0.0233, 1.5708, 0.3569],
        [0.0234, 1.5708, 0.3985]], dtype=torch.float64)

In [68]:
origin_2D = torch.reshape(origin_2D, (3,1)).float()
origin_3D = torch.reshape(origin_3D, (3,1)).float()

spherical_2D = torch.reshape(spherical_2D, (3, 349)).float()
spherical_3D = torch.reshape(spherical_3D, (3, 349)).float()

origin_2D

tensor([[0.],
        [0.],
        [0.]])

In [ ]:
origin_branch = nn.Sequential(
            nn.Conv1d(in_channels=3, out_channels=8, kernel_size=1),
            # nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.Conv1d(in_channels=8, out_channels=16, kernel_size=1),
            # nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=1),
            # nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=1),
        )

shape_branch = nn.Sequential(
            nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            # nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1),


        )

org_2 = origin_branch(origin_2D.float())
shape_2 = shape_branch(spherical_2D.float())

org_3 = origin_branch(origin_3D.float())
shape_3 = shape_branch(spherical_3D.float())

features_2 = torch.add(org_2, shape_2)
features_3 = torch.add(org_3, shape_3)

total_features = torch.vstack((features_2,features_3))
total_features.shape